<a href="https://colab.research.google.com/github/hida0/ESAA-2022-2/blob/Project/Project3_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **구글 드라이브 어마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **필요한 라이브러리 로드**

In [ ]:
import pandas as pd

## **데이터 로드**


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/22-2/ESAA/Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df.shape

(568454, 10)

In [ ]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

- 결측치 확인 결과, 사용할 4개의 칼럼(UserId, ProductId, Score, Text)에는 결측치가 없으므로 따로 전처리를 하지 않는다.

## **데이터 전처리**
- 리뷰가 너무 적은 사용자는 신뢰성이 줄어드므로, 리뷰가 일정 개수 이상인 사용자에 대해서만 추천 알고리즘을 구현한다.
- User 한 명당 리뷰 개수를 확인한 결과, 리뷰 개수가 10개 이상인 UserId를 대상으로 한다.


In [ ]:
sum(df['UserId'].value_counts()>=10)

7590

In [ ]:
# 리뷰 개수가 10개 이상인 UserId에 대한 자료만 추출
df2 = pd.DataFrame(df['UserId'].value_counts()>=10)

In [ ]:
# 리뷰 개수가 10개 이상인 사용자의 Id인 UserId만 추출
users = list(df2[df2['UserId']==True].index)
len(users)

7590

- 작성한 리뷰의 개수가 10개 이상인 7590명의 UserId 에 대하여, 고객 ID인 **`UserId`**, 상품 ID인 **`ProductId`**, 평점인 **`Score`**, 리뷰 데이터인 **`Text`** 컬럼으로 이루어진 데이터셋 **`reviews`**를 생성한다.

In [ ]:
reviews = df[ df['UserId'].isin(users) ]
reviews = reviews[ ['UserId', 'ProductId', 'Score', 'Text'] ].reset_index(drop=True)

In [ ]:
reviews.head()

,UserId,ProductId,Score,Text
0,A2725IB4YY9JEB,B0009XLVG0,5,One of my boys needed to lose some weight and ...
1,A18ECVX2RJ7HUE,B001GVISJM,4,good flavor! these came securely packed... the...
2,A2MUGFV2TDQ47K,B001GVISJM,5,The Strawberry Twizzlers are my guilty pleasur...
3,A2A9X58G2GTBLP,B001GVISJM,5,"Twizzlers, Strawberry my childhood favorite ca..."
4,AOVROBZ8BNTP7,B001EO5QW8,4,McCann's Instant Oatmeal is great if you must ...


In [ ]:
# User 한 명당 리뷰 개수 확인
reviews['UserId'].value_counts()

A3OXHLG6DIBRW8    448
A1YUL9PCJR3JTY    421
AY12DBB0U420B     389
A281NPSIMI1C2R    365
A1Z54EM24Y40LL    256
                 ... 
A3G8C334EDZPP7     10
A4P7N1ILEYDYW      10
A1M1MHI14UQSTI     10
AS8WFSJTEPKX0      10
A3IF29VV0CH70B     10
Name: UserId, Length: 7590, dtype: int64

## **VADER를 이용한 감성분석**

In [ ]:
reviews['Text'][0]

"One of my boys needed to lose some weight and the other didn't.  I put this food on the floor for the chubby guy, and the protein-rich, no by-product food up higher where only my skinny boy can jump.  The higher food sits going stale.  They both really go for this food.  And my chubby boy has been losing about an ounce a week."

In [ ]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 11.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

test = reviews['Text'][0]
senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(test)
print(senti_scores)

{'neg': 0.113, 'neu': 0.887, 'pos': 0.0, 'compound': -0.7579}


In [ ]:
senti_scores['compound']

-0.7579

In [ ]:
def vader_polarity(review, threshold=0.1):
    senti_analyzer = SentimentIntensityAnalyzer()
    scores = senti_analyzer.polarity_scores(review)
    agg_score = scores['compound']
    return agg_score

- 감성분석 수행 결과 긍정 감성 지수, 부정 감성 지수, 중립 감성 지수를 적절하게 조합한 **`compound`** 값을 추출하여 새로운 칼럼 **`vader_preds`**을 생성한다.

In [ ]:
reviews['vader_preds'] = reviews['Text'].apply( lambda x : vader_polarity(x, 0.1) )

In [ ]:
reviews

,UserId,ProductId,Score,Text,vader_preds
0,A2725IB4YY9JEB,B0009XLVG0,5,One of my boys needed to lose some weight and ...,-0.7579
1,A18ECVX2RJ7HUE,B001GVISJM,4,good flavor! these came securely packed... the...,0.9466
2,A2MUGFV2TDQ47K,B001GVISJM,5,The Strawberry Twizzlers are my guilty pleasur...,0.6486
3,A2A9X58G2GTBLP,B001GVISJM,5,"Twizzlers, Strawberry my childhood favorite ca...",0.8957
4,AOVROBZ8BNTP7,B001EO5QW8,4,McCann's Instant Oatmeal is great if you must ...,0.7103
...,...,...,...,...,...
141289,A291HTT117RVS9,B001FPT1WM,5,I've tried several violet flavored candies in ...,0.3699
141290,A1YGG6T4YJZWC1,B001FPT1WM,5,these are very pricey so i only enjoy them now...,0.9805
141291,A1YKXVGAIA8952,B003XUL27E,1,I thought this soup would be more like a chill...,0.6953
141292,A2E5C8TTAED4CQ,B001EO7N10,5,"You can make this mix yourself, but the Star A...",0.5251


## **새로운 평점 컬럼 생성**
- 기존의 평점 **`Score`**와 감성 분석으로 구한 값 **`vader_preds`**를 더하여 새로운 평점을 나타내는 칼럼 **`new_Score`**를 생성한다.

In [ ]:
reviews['new_Score'] = reviews['Score'] + reviews['vader_preds']

In [ ]:
reviews.head()

,UserId,ProductId,Score,Text,vader_preds,new_Score
0,A2725IB4YY9JEB,B0009XLVG0,5,One of my boys needed to lose some weight and ...,-0.7579,4.2421
1,A18ECVX2RJ7HUE,B001GVISJM,4,good flavor! these came securely packed... the...,0.9466,4.9466
2,A2MUGFV2TDQ47K,B001GVISJM,5,The Strawberry Twizzlers are my guilty pleasur...,0.6486,5.6486
3,A2A9X58G2GTBLP,B001GVISJM,5,"Twizzlers, Strawberry my childhood favorite ca...",0.8957,5.8957
4,AOVROBZ8BNTP7,B001EO5QW8,4,McCann's Instant Oatmeal is great if you must ...,0.7103,4.7103


### **데이터셋 복사본 생성**

In [ ]:
scores = reviews.copy()
scores.drop(['Score', 'Text', 'vader_preds'], axis=1, inplace=True)

In [ ]:
# surprise에서 사용하기 위해 컬럼명 변경
scores.columns = ['user', 'item', 'rating']
scores.head()

,user,item,rating
0,A2725IB4YY9JEB,B0009XLVG0,4.2421
1,A18ECVX2RJ7HUE,B001GVISJM,4.9466
2,A2MUGFV2TDQ47K,B001GVISJM,5.6486
3,A2A9X58G2GTBLP,B001GVISJM,5.8957
4,AOVROBZ8BNTP7,B001EO5QW8,4.7103


## **추천 시스템 구현을 위해 Surprise 데이터셋으로 로딩**

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366481 sha256=296b5f009665567a65c1e06bc0b8643261a802056ad8877723a7157f0aad2ce6
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from surprise import Reader, Dataset
from surprise.dataset import DatasetAutoFolds

- 새로운 평점 데이터는 감성 분석 데이터 결과이 더해져 범위가 -1~6이다. 이를 반영하여, `Reader` 객체를 생성할 때 `rating_sclae` 옵션에 tuple 형태로 입력해주었다.
- 전체 데이터를 학습 데이터로 사용하기 위해 `data`에 `build_full_trainset()` API를 적용하였다.

In [ ]:
reader = Reader(rating_scale=(-1.0, 6.0))
data = Dataset.load_from_df(scores, reader)

In [ ]:
# 전체 데이터를 학습 데이터로
trainset = data.build_full_trainset()

## **해당 데이터셋에서 각 알고리즘의 성능 확인**

In [ ]:
# 훈련/테스트 데이터셋 준비
from surprise.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
from surprise.prediction_algorithms import KNNBasic
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise import SVD

from surprise import accuracy

### **KNNBasic()**

In [ ]:
knn_test = KNNBasic()
knn_test.fit(train)

pred1 = knn_test.test(test)
accuracy.rmse(pred1)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0776


1.0775815959918307

### **BaselineOnly()**

In [ ]:
base_test = BaselineOnly()
base_test.fit(train)

pred2 = base_test.test(test)
accuracy.rmse(pred2)

Estimating biases using als...
RMSE: 1.1652


1.1651905507226685

### **SVD()**

In [ ]:
svd_test = SVD(biased=True)
svd_test.fit(train)

pred3 = svd_test.test(test)
accuracy.rmse(pred3)

RMSE: 1.0189


1.0189390605397546

## **추천 시스템을 구현하기 위해 필요한 함수 생성**
- 사용자가 구매한 적 없는 상품 리스트를 저장하는 함수 **`get_unpurchased`**
- 입력된 알고리즘을 이용하여, TOP-N개의 상품을 추천하는 함수 **`recommend`**

In [ ]:
def get_unpurchased(data, userId):
    # 사용자가 평점을 매긴 모든 상품 목록을 리스트로 생성
    purchased = list(set(data[data['user']== userId]['item'].tolist()))

    # 모든 상품 목록을 리스트로 생성
    total = list(set(data['item'].tolist()))

    # 모든 상품들 중 이미 평점을 매긴 상품을 제외하여 리스트로 생성
    unpurchased= list(set([x for x in total if x not in purchased]))

    print('평점 매긴 상품 수:',len(purchased), '추천 대상인 상품 수:',len(unpurchased), '전체 상품 수:',len(total))

    return unpurchased

In [ ]:
def recommend(algo, userId, unpurchased, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 상품에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(userId, productId) for productId in unpurchased]

    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]

    # top_n으로 추출된 상품 코드와 추천 예상 평점 추출
    top_items = [pred.iid for pred in top_predictions]
    top_ratings = [pred.est for pred in top_predictions]
    top_preds = [(item, rating) for item, rating in zip(top_items,  top_ratings)]

    return top_preds

## **추천 시스템 구현**
- 추천을 받을 고객의 ID를 'A18ECVX2RJ7HUE'로 선택한다.
- **`get_unpurchased`** 함수를 이용하여 해당 user가 평점을 매긴 상품 수, 추천의 대상이 되는 상품 수, 전체 상품 수를 구한다.

In [ ]:
user_id = 'A18ECVX2RJ7HUE'

In [ ]:
unpurchased = get_unpurchased(scores, user_id)

평점 매긴 상품 수: 20 추천 대상인 상품 수: 27365 전체 상품 수: 27385


### **Surprise 패키지를 이용한 최근접 이웃 방식(KNNBasic) 추천 시스템 구현**

In [ ]:
from surprise.prediction_algorithms import KNNBasic

knn = KNNBasic()
knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
top_preds = recommend(knn, user_id, unpurchased, top_n=10)

print('###{0}의 Top-10 추천 상품 리스트 ###'.format(user_id))
for product in top_preds:
    print(product[0], ":", product[1])

###A18ECVX2RJ7HUE의 Top-10 추천 상품 리스트 ###
B00113HA3A : 5.9996
B000J4IDP6 : 5.9993
B001KUWFL6 : 5.999199999999999
B005HQQSAI : 5.998999999999999
B005ISYJRY : 5.9989
B001BCUBTQ : 5.9989
B004F499PE : 5.9989
B001BCUBZ0 : 5.9989
B000CQBZRE : 5.9988
B005JDTUX6 : 5.9987


### **Surprise 패키지를 이용한 베이스라인(BaselineOnly) 추천 시스템 구현**

In [ ]:
from surprise.prediction_algorithms.baseline_only import BaselineOnly

base = BaselineOnly()
base.fit(trainset)

Estimating biases using als...


In [ ]:
top_preds = recommend(base, user_id, unpurchased, top_n=10)

print('###{0}의 Top-10 추천 상품 리스트 ###'.format(user_id))
for product in top_preds:
    print(product[0], ":", product[1])

###A18ECVX2RJ7HUE의 Top-10 추천 상품 리스트 ###
B000ED9L9E : 5.609637898728727
B001CWX7EG : 5.549320531876754
B001CWSKFC : 5.549320531876754
B000EVG8FQ : 5.549320531876754
B000EVG8HY : 5.549320531876754
B001CWV4RS : 5.549320531876754
B000EVIDWW : 5.549320531876754
B000H7ELTW : 5.52413282465422
B001PICX42 : 5.486985544913052
B000V17MLS : 5.482414417634014


### **Surprise 패키지를 이용한 잠재 요인 방식(SVD) 추천 시스템 구현**

In [ ]:
from surprise import SVD

svd = SVD(biased=True)
svd.fit(trainset)

In [ ]:
top_preds = recommend(svd, user_id, unpurchased, top_n=10)

print('###{0}의 Top-10 추천 상품 리스트 ###'.format(user_id))
for product in top_preds:
    print(product[0], ":", product[1])

###A18ECVX2RJ7HUE의 Top-10 추천 상품 리스트 ###
B001ONPMN2 : 5.867138375570361
B004OW342G : 5.758456260035832
B000G82L62 : 5.70908209601964
B000H7ELTW : 5.706900523732296
B001E5E05G : 5.703920387785711
B001EO6ESS : 5.680330138556207
B005QRPH78 : 5.6575327752757625
B001E5E1WS : 5.6519652181441655
B002N7OCGQ : 5.642107466538191
B001ONVON4 : 5.6417021526843545
